In [15]:
import os
import openai
import datetime
import pandas as pd
import json
import PyPDF2 as pdf
from PyPDF2 import PdfReader
import numpy as np
import time

In [23]:
api_key_file = open("OPENAI_API_KEY", "r")
openai.api_key = api_key_file.read()

### Dataframes

In [5]:
orders_df = pd.DataFrame({"orderID":[1,2,3], "productID":["a","b","c"],"time":[get_time(),get_time(),get_time()],"status":["delivered","pending","issue"]}).set_index("orderID")
orders_df

,productID,time,status
orderID,,,
1,a,2023-07-05 13:09:27.011995,delivered
2,b,2023-07-05 13:09:27.012031,pending
3,c,2023-07-05 13:09:27.012035,issue


In [6]:
tickets_df = pd.DataFrame({"userID":[], "ticket_details":[],"time":[]})
tickets_df

,userID,ticket_details,time


### Functions

In [4]:
def get_time():
    ct = datetime.datetime.now()
    return str(ct)

def get_status(orderID):
    return str(dict(orders_df[orders_df.index == orderID].reset_index().loc[0]))
    
    
def file_ticket(userID, ticket_details):
    global tickets_df
    tickets_df = tickets_df.append({"userID":userID, "ticket_details":ticket_details,"time":get_time()},ignore_index=True)
    return "Ticket Submitted"

### GPT Code

In [43]:
premise = "You’re a kind helpful tech support chatbot. For ticket filings: Ensure that the user provides their UserID when asking for details. If they do not provide their ID, do not let them file a ticket. When the ticket is filed successfuly, do not provide advice."
premise = premise
messages=[
    {"role": "system", "content": premise}
      ]
functions=[{
                "name": "get_time",
                "description": "returns string representation of a datetime object. For reference, this is in Singapore, so be able to guess the time in other time zones when a different country is specified",
                "parameters": {
                    "type": "object",
                    "properties": {
                    },
                    "required": [],
                },
            },
    {
                "name": "file_ticket",
                "description": "files a ticket to a dataframe, logging UserID and ticket_details.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "userID": {"type": "integer"},
                        "ticket_details": {"type": "string"},
                    },
                    "required": ["userID","ticket_details"],
                },
            },
    
    {
                "name": "get_status",
                "description": "gets the status in dictionary form of an order of a specific order ID",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "orderID": {"type": "integer"},
                    },
                    "required": ["orderID"],
                },
            }
]



def conversation(response):
    content = response
    messages.append({"role": "user", "content": content})
    
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions, function_call = "auto",
    )
    message = completion["choices"][0]["message"]
    
    #Functions Block
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        print(message["function_call"]["arguments"])
        arguments = json.loads(message["function_call"]["arguments"])
        
        
        #Time Function
        if function_name == "get_time":
            function_response = get_time()
            
        #Ticket Function
        elif function_name == "file_ticket":
            function_response = file_ticket(arguments["userID"],arguments["ticket_details"])
        
        #Order Status
        elif function_name == "get_status":
            function_response = get_status(arguments["orderID"])
            
        #Error Handling
        else:
            raise NotImplementedError()
        
        messages.append({"role":"function","name":function_name,"content":function_response})
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages = messages,
        )
        answer_response = second_response.choices[0]["message"]["content"].strip()
        messages.append({"role": "assistant", "content": answer_response})
    
    
    #Normal Block
    else:
        chat_response = completion.choices[0].message.content
        messages.append({"role": "assistant", "content": chat_response})
    time.sleep(15)
    for i in messages:
        if i['role'] == "user":
            print("User: ", i['content'])
            #print("\n")
        elif i['role'] == "assistant":
            print("ChatGPT: ", i['content'])
        print("\n")
    

In [52]:
conversation("Before I go, what time is it at the moment")



{}


User:  Hi I'd like to file a ticket. My monitor keeps flickering and shows rainbows and blacks


ChatGPT:  Sure, I can help you with that. Before we proceed, can you please provide me with your UserID?


User:  it is 17823213




ChatGPT:  Thank you for providing your UserID. I have successfully filed a ticket for you regarding the issue with your flickering monitor. Our support team will look into it and assist you soon. Please note that as a tech support chatbot, I am unable to provide advice. Is there anything else I can assist you with?


User:  Yes, I'd also like to check the status of one of my orders


ChatGPT:  Sure, I can help you with that. Can you please provide me with the order ID of the order you would like to check the status of?


User:  Of course, my order ID is 3




ChatGPT:  Thank you for providing the order ID. I see that there is an issue with your order. I will escalate this to our support team and they will look into it. They will reach out to you soon to p

In [53]:
tickets_df

,userID,ticket_details,time
0,17823213.0,Monitor keeps flickering and showing rainbows ...,2023-07-05 13:30:15.738382
1,17823213.0,There is an issue with order ID 3.,2023-07-05 13:33:02.463881
